# Planes Intersection

## Packages:

In [112]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import numpy as np

## Utilities:

In [113]:
def mesh2D(xlim, ylim, n=5): #Set up Meshgrid
    if isinstance(n, int):
        x = np.linspace(xlim[0],xlim[1],n)
        y = np.linspace(ylim[0],ylim[1],n)
    elif isinstance(n, list):
        x = np.linspace(xlim[0],xlim[1],n[0])
        y = np.linspace(ylim[0],ylim[1],n[1])
    else:
        raise Exception("Invalid Parameter")
        
    return np.meshgrid(x, y, sparse=True)

In [114]:
def normalize(v): #for Evec, Eval
    magnitude = np.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
    if magnitude==0:
        raise ValueError("Zero vector cannot be normalized.")
    else:
        return v/magnitude

## Objects:

In [115]:
class Line: 
    def __init__(self, vec, offset):
        self.vec = vec
        
        t = np.linspace(0,1,6) #parameter
        self.x = offset[0] + t*vec[0] #x points
        self.y = offset[1] + t*vec[1] #y points
        self.z = offset[2] + t*vec[2] #z points
        
    def gObject(self, layout=None):
        lineObject = go.Scatter3d(mode="lines",
                                  x=self.x,
                                  y=self.y,
                                  z=self.z,
                                  line=dict(color=('rgb(210,64,0)'),
                                            width=7)
                                 )
        return lineObject

In [116]:
class Sphere:
    def __init__(self, radius=5, center=[0, 0, 0]):
        self.radius = radius
        self.center = center
        meshSize = 20
        theta = np.linspace(0,2*np.pi,meshSize)
        phi = np.linspace(0,np.pi,meshSize)
        self.x = radius*np.outer(np.cos(theta),np.sin(phi)) + center[0]
        self.y = radius*np.outer(np.sin(theta),np.sin(phi)) + center[1]
        self.z = radius*np.outer(np.ones(meshSize),np.cos(phi)) + center[2]
        
    def gObject(self):
        sphere = go.Surface(name='Sphere',
                            x=self.x,
                            y=self.y,
                            z=self.z,
                            showscale=False,
                            opacity=0.5,
                            colorscale=[[0.0, 'rgb(2.137.59)'],
                                        [1.0, 'rgb(0,62,116)']
                                       ]
                            )
        return sphere

In [117]:
class Point:
    def __init__(self, position):
        self.position = np.array(position)
    
    def gObject(self):
        point = go.Scatter3d(mode="markers",
                             x=[self.position[0]],
                             y=[self.position[1]],
                             z=[self.position[2]],
                             marker=dict(color='rgb(12,161,205)',
                                         size=7
                                        )
                            )
        return point

### 3D Visualization

In [118]:
line1 = Line([1, 0, 0], [0, 0, 0])
line2 = Line([0, 1, 0], [0, 0, 0])
line3 = Line([0, 0, 1], [0, 0, 0])
sphere1 = Sphere(2*np.sqrt(3))
point1 = Point([2,2,2])

data = [line1.gObject(),
        line2.gObject(),
        line3.gObject(),
        sphere1.gObject(),
        point1.gObject()
       ]

In [119]:
#fig=go.Figure(data=data)
#py.plot(fig)

In [120]:
theta1 = np.pi/2
theta2 = -np.pi/2
t1 = np.linspace(0, theta1, 10)
t2 = np.linspace(0, theta2, 10)

init = np.matrix([[2], [2], [2]])


for i in t1:
    Rz = np.matrix([[np.cos(i), -np.sin(i), 0], [np.sin(i), np.cos(i), 0], [0, 0 ,1]])
    p1 = Rz*init
    traP1 = np.reshape(p1, (1,3))
    traP1list = traP1.tolist()[0]
    point2 = Point(traP1list)
    data.append(point2.gObject())
    
for i in t1:
    Rx = np.matrix([[1, 0, 0], [0, np.cos(i), -np.sin(i)], [0, np.sin(i), np.cos(i)]])
    p2 = Rx*p1
    traP2 = np.reshape(p2, (1,3))
    traP2list = traP2.tolist()[0]
    point3 = Point(traP2list)
    data.append(point3.gObject())
    
for i in t1:
    Rx = np.matrix([[1, 0, 0], [0, np.cos(i), -np.sin(i)], [0, np.sin(i), np.cos(i)]])
    p3 = Rx*init
    traP3 = np.reshape(p3, (1,3))
    traP3list = traP3.tolist()[0]
    point4 = Point(traP3list)
    data.append(point4.gObject())
    

for i in t1:
    Rz = np.matrix([[np.cos(i), -np.sin(i), 0], [np.sin(i), np.cos(i), 0], [0, 0 ,1]])
    p4 = Rz*p3
    traP4 = np.reshape(p4, (1,3))
    traP4list = traP4.tolist()[0]
    point5 = Point(traP4list)
    data.append(point5.gObject())

    

    


In [121]:
fig=go.Figure(data=data)
py.plot(fig)

'file:///Users/rohankamath/Desktop/UROP /Thursday 3D Visualizations/temp-plot.html'